In [1]:
import os
from  dotenv import load_dotenv

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Traditional RAG"

In [3]:
from langchain_community.document_loaders import TextLoader

In [4]:
loader = TextLoader("blog.txt")
doc = loader.load()

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap=20)

In [7]:
split_doc=splitter.split_documents(doc)

In [8]:
from langchain_openai import OpenAIEmbeddings

In [9]:
openAI_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
import faiss

In [ ]:
from langchain_community.vectorstores import FAISS

In [ ]:
index=faiss.IndexFlatL2(1536)

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [ ]:
index = faiss.IndexFlatL2(1536)
vector_store= FAISS(
    openAI_embeddings,
    index,
    docstore=InMemoryDocstore({}),
    index_to_docstore_id={}
)

In [ ]:
vector_store.add_documents(documents= split_doc)

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":10}
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the information provided in the context below. "
    "If the answer is not contained in the context, respond with 'I don't know.'\n\n"
    "Context:\n{context}\n\nQuestion:\n{question}"
)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
output_parser = StrOutputParser()

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0
)

In [ ]:
def join_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])


In [ ]:
from langchain.schema.runnable import RunnablePassthrough

In [ ]:
rag_chain = (
    {"context": retriever | join_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [ ]:
question = "What is the main topic of the blog?"
result = rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {result}")
print()

